In [1]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost as xgb
import warnings
warnings.simplefilter("ignore")

In [2]:
X = pd.read_csv("/Users/atul/Library/CloudStorage/OneDrive-Personal/Stevens Institute of Technology/Courses/CS 513/Project/Datasets/Credit score classification/train.csv")
y = pd.read_csv("/Users/atul/Library/CloudStorage/OneDrive-Personal/Stevens Institute of Technology/Courses/CS 513/Project/Datasets/Credit score classification/test.csv")

In [3]:
X.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [4]:
y

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x160a,CUS_0xd40,September,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,2022.0,Good,809.98,35.030402,22 Years and 9 Months,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,0x160b,CUS_0xd40,October,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.053114,22 Years and 10 Months,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
2,0x160c,CUS_0xd40,November,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.811894,NaN,No,49.574949,148.23393788500925,Low_spent_Medium_value_payments,264.67544623342997
3,0x160d,CUS_0xd40,December,Aaron Maashoh,24_,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,32.430559,23 Years and 0 Months,No,49.574949,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,0x1616,CUS_0x21b1,September,Rick Rothackerj,28,004-07-5839,_______,34847.84,3037.986667,2,...,5.0,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018417945296,High_spent_Large_value_payments,485.2984336755923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0x25fe5,CUS_0x8600,December,Sarah McBridec,4975,031-35-0942,Architect,20002.88,1929.906667,10,...,12.0,_,3571.7,34.780553,NaN,Yes,60.964772,146.48632477751087,Low_spent_Small_value_payments,275.53956951573343
49996,0x25fee,CUS_0x942c,September,Nicks,25,078-73-5990,Mechanic,39628.99,NaN,4,...,7.0,Good,502.38,27.758522,31 Years and 11 Months,NM,35.104023,181.44299902757518,Low_spent_Small_value_payments,409.39456169535066
49997,0x25fef,CUS_0x942c,October,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,7.0,Good,502.38,36.858542,32 Years and 0 Months,No,35.104023,__10000__,Low_spent_Large_value_payments,349.7263321025098
49998,0x25ff0,CUS_0x942c,November,Nicks,25,078-73-5990,Mechanic,39628.99,NaN,4,...,7.0,Good,502.38,39.139840,32 Years and 1 Months,No,35.104023,97.59857973344877,High_spent_Small_value_payments,463.23898098947717


In [5]:
min_age = 18
max_age = 100
max_num_credit_cards = 10
max_interest_rate = 50
max_num_loans = 10
max_num_credit_inquiries = 50

def find_latest(x):
    for value in reversed(x):
        if pd.notna(value):
            return value
    return None

def remove_underscore(value):
    if isinstance(value, str):
        return value.replace('_', '')
    return value

def check_valid_age(value):
    if value >= min_age and value <= max_age:
        return value
    return None

def check_num_credit_cards(value):
    if value <= max_num_credit_cards:
        return value
    return None

def check_interest_rate(value):
    if value <= max_interest_rate:
        return value
    return None

def check_max_num_loan(value):
    if value >= 0 and value <= max_num_loans:
        return value
    return None

def check_num_delayed_payments(value):
    if isinstance(value, int) and value >= 0:
        return value
    return None

def check_num_credit_inquiries(value):
    if value <= max_num_credit_inquiries:
        return value
    return None

def check_outstanding_debt(value):
    if value >= 0:
        return value
    return None

def check_credit_history_age(value):
    match = re.match(r'(\d+) Years and (\d+) Months', value)

    if match:
        years = int(match.group(1))
        months = int(match.group(2))
        total_months = years * 12 + months
        return total_months
    return None

def divide_loan_column(X):
    loan_unique = X["Type_of_Loan"].unique()
    type_of_loan = []

    for item in loan_unique:
        item_list = item.split(',')
        for loan in item_list:
            loan = loan.replace(' ','')
            loan = loan.replace('and','')
            if loan not in type_of_loan:
                type_of_loan.append(loan)
            
    type_of_loan.remove('nan')
    print(type_of_loan)

    X[type_of_loan] = 0

    for index, row in X.iterrows():
        item = row["Type_of_Loan"]
        item_list = item.split(',')
        for loan in item_list:
            loan = loan.replace(' ','')
            loan = loan.replace('and','')
            X.at[index,loan] = 1

    X.drop('Type_of_Loan',axis=1,inplace=True)
    return X

def preprocessing(data):
    data.drop_duplicates()
    
    integer_columns = ['Age','Num_Bank_Accounts','Num_Credit_Card','Interest_Rate','Num_of_Loan','Delay_from_due_date',
                      'Num_of_Delayed_Payment','Num_Credit_Inquiries']
    float_columns = ['Annual_Income','Monthly_Inhand_Salary','Changed_Credit_Limit','Outstanding_Debt','Credit_Utilization_Ratio',
                    'Total_EMI_per_month','Amount_invested_monthly','Monthly_Balance']
    
    data = data.astype(str)
    data = data.applymap(remove_underscore)
    
    for column in integer_columns:
        data[column] = pd.to_numeric(data[column], errors='coerce').astype('Int64')
    
    for column in float_columns:
        data[column] = pd.to_numeric(data[column], errors='coerce').astype('float64')

    data['Age'] = data['Age'].apply(check_valid_age).astype('Int64')
    data['Num_Credit_Card'] = data['Num_Credit_Card'].apply(check_num_credit_cards).astype('Int64')
    data['Interest_Rate'] = data['Interest_Rate'].apply(check_interest_rate).astype('Int64')
    data['Num_of_Delayed_Payment'] = data['Num_of_Delayed_Payment'].apply(check_num_delayed_payments).astype('Int64')
    data['Num_Credit_Inquiries'] = data['Num_Credit_Inquiries'].notna().apply(check_num_credit_inquiries).astype('Int64') #FIX the boolean value issue
    #data['Outstanding_Debt'] = data['Outstanding_Debt'].apply(check_num_delayed_payments).astype('float64')
    data['Credit_History_Age'] = data['Credit_History_Age'].apply(check_credit_history_age).astype('Int64')
    return data


In [6]:
X = preprocessing(X)
y = preprocessing(y)
#X.to_csv("/Users/atul/Library/CloudStorage/OneDrive-Personal/Stevens Institute of Technology/Courses/CS 513/Project/Datasets/Credit score classification/new.csv")

In [7]:
# Not using the columns: Num_Bank_Accounts, Num_Credit_Cards, Changed_Credit_Limit

def group_data(data):
    grouped_data = X.groupby('Customer_ID').agg({
    'Name': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'Age': lambda x: x.iloc[-1] if not x.empty else None,
    'Occupation': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'Annual_Income': lambda x: x.iloc[-1] if not x.empty else None,
    'Monthly_Inhand_Salary': lambda x: x.iloc[-1] if not x.empty else None,
    'Interest_Rate': lambda x: x.iloc[-1] if not x.empty else None,
    'Num_of_Loan': lambda x: x.iloc[-1] if not x.empty else None,
    'Type_of_Loan': lambda x: x.iloc[-1] if not x.empty else None,
    'Delay_from_due_date': lambda x: x.dropna().mean(),
    'Num_of_Delayed_Payment': lambda x: x.dropna().sum(),
    'Num_Credit_Inquiries': lambda x: x.iloc[-1] if not x.empty else None,
    'Credit_Mix' : lambda x: x.iloc[-1] if not x.empty else None,
    'Outstanding_Debt': lambda x: x.iloc[-1] if not x.empty else None,
    'Credit_Utilization_Ratio': lambda x: x.iloc[-1] if not x.empty else None,
    'Credit_History_Age': lambda x: x.iloc[-1] if not x.empty else None,
    'Payment_of_Min_Amount': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'Total_EMI_per_month': lambda x: x.iloc[-1] if not x.empty else None,
    'Amount_invested_monthly': lambda x: x.dropna().mean(),
    'Payment_Behaviour': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'Monthly_Balance': lambda x: x.dropna().mean(),
    'Credit_Score': lambda x: x.iloc[-1] if not x.empty else None}).reset_index()
    
    return grouped_data

grouped_X = group_data(X)
grouped_y = group_data(y)
grouped_X = divide_loan_column(grouped_X)
grouped_y = divide_loan_column(grouped_y)
grouped_X

,Customer_ID,Name,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,CUS0x1000,Alistair Barrf,18,Lawyer,30625.940,2706.161667,27,2,"Credit-Builder Loan, and Home Equity Loan",62.250,...,Bad,1562.91,30.077191,129,Yes,42.941090,158.549735,HighspentMediumvaluepayments,335.375341,Poor
1,CUS0x1009,Arunah,26,Mechanic,52312.680,4250.390000,17,4,"Not Specified, Home Equity Loan, Credit-Builde...",7.250,...,,202.68,40.286997,372,Yes,108.366467,146.679378,HighspentMediumvaluepayments,428.743155,Standard
2,CUS0x100b,Shirboni,19,MediaManager,113781.390,9549.782500,1,0,nan,13.500,...,Good,1030.20,28.592943,190,No,0.000000,1861.759880,HighspentLargevaluepayments,781.229776,Standard
3,CUS0x1011,Schneyerh,44,Doctor,58918.470,5208.872500,17,3,"Student Loan, Credit-Builder Loan, and Debt Co...",27.250,...,Standard,473.14,27.829959,190,Yes,123.434939,1522.178300,LowspentMediumvaluepayments,332.642837,Standard
4,CUS0x1013,Cameront,44,Mechanic,98620.980,NaN,6,3,"Student Loan, Debt Consolidation Loan, and Per...",12.500,...,Good,1233.51,26.524864,214,No,228.018084,355.442408,HighspentMediumvaluepayments,472.781009,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS0xff3,Somervilled,55,Scientist,17032.785,1176.398750,2,3,"Personal Loan, Mortgage Loan, and Auto Loan",12.250,...,Good,1229.08,26.915577,<NA>,No,33.299764,83.918549,LowspentSmallvaluepayments,271.671562,Standard
12496,CUS0xff4,Poornimaf,37,Entrepreneur,25546.260,2415.855000,14,5,"Not Specified, Student Loan, Student Loan, Cre...",15.875,...,Standard,758.44,39.333488,225,Yes,101.328637,152.775690,LowspentSmallvaluepayments,259.981173,Standard
12497,CUS0xff6,Shieldsb,19,Doctor,117639.920,9727.326667,1,2,"Home Equity Loan, and Auto Loan",-0.125,...,Good,338.30,32.871853,299,No,126.638453,1990.195603,HighspentSmallvaluepayments,667.322417,Good
12498,CUS0xffc,Brads,18,Musician,60877.170,5218.097500,27,8,"Credit-Builder Loan, Payday Loan, Not Specifie...",45.375,...,,1300.13,29.026498,<NA>,Yes,272.809169,1386.651352,HighspentLargevaluepayments,339.951771,Poor


In [8]:
grouped_y

,Customer_ID,Name,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,CUS0x1000,Alistair Barrf,18,Lawyer,30625.940,2706.161667,27,2,"Credit-Builder Loan, and Home Equity Loan",62.250,...,Bad,1562.91,30.077191,129,Yes,42.941090,158.549735,HighspentMediumvaluepayments,335.375341,Poor
1,CUS0x1009,Arunah,26,Mechanic,52312.680,4250.390000,17,4,"Not Specified, Home Equity Loan, Credit-Builde...",7.250,...,,202.68,40.286997,372,Yes,108.366467,146.679378,HighspentMediumvaluepayments,428.743155,Standard
2,CUS0x100b,Shirboni,19,MediaManager,113781.390,9549.782500,1,0,nan,13.500,...,Good,1030.20,28.592943,190,No,0.000000,1861.759880,HighspentLargevaluepayments,781.229776,Standard
3,CUS0x1011,Schneyerh,44,Doctor,58918.470,5208.872500,17,3,"Student Loan, Credit-Builder Loan, and Debt Co...",27.250,...,Standard,473.14,27.829959,190,Yes,123.434939,1522.178300,LowspentMediumvaluepayments,332.642837,Standard
4,CUS0x1013,Cameront,44,Mechanic,98620.980,NaN,6,3,"Student Loan, Debt Consolidation Loan, and Per...",12.500,...,Good,1233.51,26.524864,214,No,228.018084,355.442408,HighspentMediumvaluepayments,472.781009,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS0xff3,Somervilled,55,Scientist,17032.785,1176.398750,2,3,"Personal Loan, Mortgage Loan, and Auto Loan",12.250,...,Good,1229.08,26.915577,<NA>,No,33.299764,83.918549,LowspentSmallvaluepayments,271.671562,Standard
12496,CUS0xff4,Poornimaf,37,Entrepreneur,25546.260,2415.855000,14,5,"Not Specified, Student Loan, Student Loan, Cre...",15.875,...,Standard,758.44,39.333488,225,Yes,101.328637,152.775690,LowspentSmallvaluepayments,259.981173,Standard
12497,CUS0xff6,Shieldsb,19,Doctor,117639.920,9727.326667,1,2,"Home Equity Loan, and Auto Loan",-0.125,...,Good,338.30,32.871853,299,No,126.638453,1990.195603,HighspentSmallvaluepayments,667.322417,Good
12498,CUS0xffc,Brads,18,Musician,60877.170,5218.097500,27,8,"Credit-Builder Loan, Payday Loan, Not Specifie...",45.375,...,,1300.13,29.026498,<NA>,Yes,272.809169,1386.651352,HighspentLargevaluepayments,339.951771,Poor


In [10]:
# Normalizing the dataset

def normalize_data(data):
    data.dropna(inplace=True)
    data.drop('Customer_ID', axis=1, inplace=True)
    data.drop('Name', axis=1, inplace=True)

    categorical_cols = ['Credit_Mix','Occupation','Payment_of_Min_Amount','Payment_Behaviour', 'Credit_Score']

    data_cols = ['Age','Interest_Rate','Num_of_Loan','Delay_from_due_date',
             'Num_of_Delayed_Payment','Num_Credit_Inquiries','Annual_Income','Monthly_Inhand_Salary',
             'Outstanding_Debt','Credit_Utilization_Ratio','Credit_History_Age','Total_EMI_per_month',
             'Amount_invested_monthly','Monthly_Balance']

    label_encoder = LabelEncoder()
    scaler = MinMaxScaler()

    for column in categorical_cols:
        data[column] = label_encoder.fit_transform(data[column])
    
    for column in data_cols:
        data[column] = scaler.fit_transform(data[column].values.reshape(-1,1))
    
    return data
    
normalized_X = normalize_data(grouped_X)
normalized_y = normalize_data(grouped_y)

normalized_X

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Mix,...,Credit-BuilderLoan,HomeEquityLoan,NotSpecified,PaydayLoan,StudentLoan,DebtConsolidationLoan,PersonalLoan,AutoLoan,MortgageLoan,nan
2,0.026316,11,0.005088,0.629446,0.000000,0.063371,0.446640,0.008338,1.0,1,...,0,0,0,0,0,0,0,0,0,1.0
5,0.236842,7,0.001817,0.222714,0.789474,0.063371,0.272727,0.010317,1.0,2,...,0,0,0,0,0,0,0,0,0,1.0
15,0.684211,6,0.001156,0.154500,0.000000,0.063371,0.083004,0.004522,1.0,0,...,0,0,0,0,0,0,0,0,0,1.0
23,0.157895,1,0.001204,0.122432,0.368421,0.063371,0.142292,0.012012,1.0,0,...,0,0,0,0,0,0,0,0,0,1.0
38,0.605263,7,0.000381,0.055066,0.157895,0.063371,0.304348,0.007914,1.0,0,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12362,0.157895,6,0.000569,0.074332,0.421053,0.063371,0.830040,0.014274,1.0,2,...,0,0,0,0,0,0,0,0,0,1.0
12394,0.105263,13,0.004809,0.572765,0.421053,0.063371,0.928854,0.015828,1.0,2,...,0,0,0,0,0,0,0,0,0,1.0
12464,1.000000,11,0.008134,1.000000,0.578947,0.063371,0.272727,0.004664,1.0,1,...,0,0,0,0,0,0,0,0,0,1.0
12481,0.078947,1,0.001583,0.218052,0.315789,0.063371,0.067194,0.013708,1.0,0,...,0,0,0,0,0,0,0,0,0,1.0


In [11]:
normalized_y

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Mix,...,Credit-BuilderLoan,HomeEquityLoan,NotSpecified,PaydayLoan,StudentLoan,DebtConsolidationLoan,PersonalLoan,AutoLoan,MortgageLoan,nan
2,0.026316,11,0.005088,0.629446,0.000000,0.063371,0.446640,0.008338,1.0,1,...,0,0,0,0,0,0,0,0,0,1.0
5,0.236842,7,0.001817,0.222714,0.789474,0.063371,0.272727,0.010317,1.0,2,...,0,0,0,0,0,0,0,0,0,1.0
15,0.684211,6,0.001156,0.154500,0.000000,0.063371,0.083004,0.004522,1.0,0,...,0,0,0,0,0,0,0,0,0,1.0
23,0.157895,1,0.001204,0.122432,0.368421,0.063371,0.142292,0.012012,1.0,0,...,0,0,0,0,0,0,0,0,0,1.0
38,0.605263,7,0.000381,0.055066,0.157895,0.063371,0.304348,0.007914,1.0,0,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12362,0.157895,6,0.000569,0.074332,0.421053,0.063371,0.830040,0.014274,1.0,2,...,0,0,0,0,0,0,0,0,0,1.0
12394,0.105263,13,0.004809,0.572765,0.421053,0.063371,0.928854,0.015828,1.0,2,...,0,0,0,0,0,0,0,0,0,1.0
12464,1.000000,11,0.008134,1.000000,0.578947,0.063371,0.272727,0.004664,1.0,1,...,0,0,0,0,0,0,0,0,0,1.0
12481,0.078947,1,0.001583,0.218052,0.315789,0.063371,0.067194,0.013708,1.0,0,...,0,0,0,0,0,0,0,0,0,1.0


In [12]:
# Splitting the target variable
X_train = normalized_X.drop('Credit_Score',axis=1)
y_train = normalized_X['Credit_Score']
X_test = normalized_y.drop('Credit_Score',axis=1)
y_test = normalized_y['Credit_Score']

In [13]:
# Implementing Logistic Regression

lr_model = LogisticRegression(penalty='l2',random_state=42)

# Train the model on the training data
lr_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = lr_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(accuracy)
print(conf_matrix)
print(classification_rep)

0.5890944498539435
[[129   0 192]
 [ 25   1 123]
 [ 82   0 475]]
              precision    recall  f1-score   support

           0       0.55      0.40      0.46       321
           1       1.00      0.01      0.01       149
           2       0.60      0.85      0.71       557

    accuracy                           0.59      1027
   macro avg       0.72      0.42      0.39      1027
weighted avg       0.64      0.59      0.53      1027



In [14]:
# svm_model = SVC(kernel='linear', random_state=42)

# # Train the model on the training data
# svm_model.fit(X_train, y_train)

# # Make predictions on the testing data
# y_pred = svm_model.predict(X_test)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# conf_matrix = confusion_matrix(y_test, y_pred)
# classification_rep = classification_report(y_test, y_pred)

# print(f'Accuracy: {accuracy:.2f}')
# print('\nConfusion Matrix:\n', conf_matrix)
# print('\nClassification Report:\n', classification_rep)

In [15]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data
rf_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('\nConfusion Matrix:\n', conf_matrix)
print('\nClassification Report:\n', classification_rep)

Accuracy: 1.00

Confusion Matrix:
 [[321   0   0]
 [  0 149   0]
 [  0   0 557]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       321
           1       1.00      1.00      1.00       149
           2       1.00      1.00      1.00       557

    accuracy                           1.00      1027
   macro avg       1.00      1.00      1.00      1027
weighted avg       1.00      1.00      1.00      1027



In [16]:
# Instantiate XGBoost classifier
xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Fit the model
xgb_classifier.fit(X_train, y_train)

# Make predictions
y_pred = xgb_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 1.0
